In [3]:
import pandas as pd


In [4]:
orders = pd.read_csv("data/orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [5]:
users = pd.read_json("data/users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
import sqlite3

conn = sqlite3.connect("data/restaurants.db")

with open("data/restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

conn.close()

In [8]:
conn = sqlite3.connect("data/restaurants.db")

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [9]:
final_df = orders.merge(users, on="user_id", how="left")
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [10]:
final_df = final_df.merge(restaurants, on="restaurant_id", how="left")
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [11]:
orders.shape
final_df.shape

(10000, 12)

In [12]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [13]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df.groupby(final_df["order_date"].dt.month)["order_id"].count()

C:\Users\JANHVI\AppData\Local\Temp\ipykernel_12932\1390911509.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


order_date
1     831
2     785
3     903
4     812
5     844
6     784
7     859
8     851
9     812
10    863
11    807
12    849
Name: order_id, dtype: int64

In [14]:
final_df.groupby("membership")["total_amount"].agg(["count", "mean", "sum"])


,count,mean,sum
membership,,,
Gold,4987,797.145556,3975364.89
Regular,5013,805.158434,4036259.23


In [15]:
final_df.groupby("city")["total_amount"].sum()


city
Bangalore    2206946.58
Chennai      1990513.03
Hyderabad    1889366.58
Pune         1924797.93
Name: total_amount, dtype: float64

In [16]:
final_df.groupby("cuisine")["total_amount"].sum()

cuisine
Chinese    1930504.65
Indian     1971412.58
Italian    2024203.80
Mexican    2085503.09
Name: total_amount, dtype: float64

In [17]:
final_df[final_df["rating"] >= 4.5].shape[0]

3374